<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/bi-2021/blob/main/materialy-wyklady/bi_2021_04_19_chen_li_wu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Wprowadzenie

W tym notatniku pokażę w jaki sposób estymować parametry wykorzystując podejście zaprezentowane w pracy:

Chen, Y., Li, P., & Wu, C. (2020). **Doubly Robust Inference With Nonprobability Survey Samples**. Journal of the American Statistical Association, 115(532), 2011–2021. https://doi.org/10.1080/01621459.2019.1677241

Zanim jednak zacznę omawiać jak należy do tego podejść zainstalujemy wymagane pakiety: `survey` oraz `rootSolve`. 


In [1]:
install.packages(c("survey", "rootSolve"))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘minqa’, ‘numDeriv’, ‘mitools’




Ładujemy pakiety oraz wczytujemy dane

In [22]:
library(survey) 
library(rootSolve)
library(tidyverse)
library(Matrix) ## macierze rzadkie, wprowadzimy je później

In [3]:
download.file("https://raw.githubusercontent.com/DepartmentOfStatisticsPUE/bi-2021/main/data/popyt-zajecia-dane.csv",
              "popyt-dane.csv")
dane <- read.csv("popyt-dane.csv", stringsAsFactors = FALSE)
head(dane)              

,id_popyt,id_jednostki,waga,sek,klasa_pr,sekc_pkd,woj,zawod_kod2,wolne_miejsca,id_cbop,jedna_zmiana,wymiar_40,wolne_miejsca_cbop,wolne_miejsca_niepeln_cbop
,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,<lgl>,<lgl>,<int>,<int>
1,1,a9cc990df6a99ab215a1bc13f51d4825c7d52d18,1,1,D,O,14,1,2,NA,NA,NA,NA,NA
2,2,a9cc990df6a99ab215a1bc13f51d4825c7d52d18,1,1,D,O,14,2,7,NA,NA,NA,NA,NA
3,3,c9dbaf50890165ebe810aa770de0e9df903dc35b,6,1,D,O,24,2,6,NA,NA,NA,NA,NA
4,4,718e0bba42bcec6ed98f9690db6d26cb7b93c880,1,1,D,R.S,14,2,7,NA,NA,NA,NA,NA
5,5,532a1879a692b9d7bbb7282ba757d028156ef341,1,1,D,R.S,14,2,6,NA,NA,NA,NA,NA
6,6,0b6b623fa45e257284a3049d097af322841337e3,1,1,D,R.S,22,2,1,NA,NA,NA,NA,NA


Na potrzeby tego przykładu przyjmiemy uproszczone podejście tj.

+ zakładamy, że dwa zbiory danych są rozłączne,
+ skupimy się wyłącznie na podmiotach gospodarczych,
+ cechą, która nas interesuje to odsetek ofert, które dotyczą pracy na jedną zmianę.

Czyli agregujemy dane na poziom podmiotów gospodarczych 


In [16]:
proba_losowa <- dane %>%
  ungroup() %>%
  filter(!is.na(id_popyt)) %>%
  distinct(id_jednostki, waga, sek, wielk = klasa_pr, pkd = sekc_pkd, woj) 

proba_nielosowa <- dane %>%
  ungroup() %>%
  filter(!is.na(id_cbop)) %>%
  group_by(id_jednostki, sek, wielk = klasa_pr, pkd = sekc_pkd, woj) %>%
  summarise(odsetek = weighted.mean(jedna_zmiana, wolne_miejsca_cbop))

`summarise()` has grouped output by 'id_jednostki', 'sek', 'wielk', 'pkd'. You can override using the `.groups` argument.



In [17]:
## liczebność próby losowej
nrow(proba_losowa)

## liczebnosć próby
nrow(proba_nielosowa)

[1] 6523

[1] 9344

Łączymy dane w jeden zbiór danych

In [165]:
dane_estymacja <- bind_rows(
  proba_nielosowa %>% 
      ungroup()  %>%
      mutate(proba = 1, waga = 1, sek = factor(sek), woj = factor(woj)),
  proba_losowa %>% 
    ungroup()  %>%
    mutate(proba = 0, sek = factor(sek), woj = factor(woj))
)

head(dane_estymacja)

id_jednostki,sek,wielk,pkd,woj,odsetek,proba,waga
<chr>,<fct>,<chr>,<chr>,<fct>,<dbl>,<dbl>,<dbl>
000167228537c46b1b084e9c501b065ca42374ce,1,M,Q,8,1,1,1
0001d7cc2ffede0b1bde90bd4441d63fda57bcf0,2,M,G,12,0,1,1
000be1bfb49ae8082ef18d325c2ca6a7aeb0efe6,2,M,C,22,0,1,1
000f0546b2f54d82de8efd89612c2cb9e8728112,2,S,P,12,1,1,1
001a4ae4b2153adce49fa5a77825990cbd008419,2,D,G,24,0,1,1
0022b78360fff23fb836b05a93cffafa1d951593,2,D,N,4,0,1,1


## Metodyka

### Funkcja wiarygodności 

W skrócie, Chen, Li i Wu (2020) zaproponowali następującą logarytm pseudo-funkcji wiarygodności:

$$
\log L^* = \sum_{i \in S_A} \mathbf{x}_i^T \mathbf{\theta} - \sum_{i \in S_B} d_i^B \log \{ 1 + \exp(\mathbf{x}_i^T \mathbf{\theta}) \},
$$

gdzie $S_A$ to próba nielosowa, $S_B$ to próba losowa, $\mathbf{x}_i$ to wektor zmiennych, które wykorzystujemy w modelu, $\mathbf{\theta}$ to wektor parametrów funkcji *propensity score* (w tym przypadku zastosowano regresję logistyczną; uwaga: powyższa funkcja jest wynikiem przeksztalceń dlatego nigdzie nie ma standardowego zapisu $\exp(\mathbf{x}_i^T \mathbf{\theta})/(1 + \exp(\mathbf{x}_i^T \mathbf{\theta}))$), $d_i^B$ to wagi wynikajace z losowania (lub wagi finalne) przypisane każdej jednostce $i$ w próbie losowej.

Aby znaleźć wektor parametrów $\mathbf{\theta}$, który maksymalizuje $\log L^*$ należy albo zastosować procedurę optymalizacyjną albo rozwiązać następujący układ równań (tzw. score equations):

$$
U(\mathbf{\theta}) = \frac{\partial \log L^*(\mathbf{\theta})}{\partial \mathbf{\theta}} = \sum_{i \in S_A} \mathbf{x}_i - \sum_{i \in S_B} d_i^B \pi(\mathbf{x}_i, \mathbf{\theta})\mathbf{x}_i,
$$

gdzie $\pi(\mathbf{x}_i, \mathbf{\theta}) = \exp(\mathbf{x}_i^T \mathbf{\theta})/(1 + \exp(\mathbf{x}_i^T \mathbf{\theta}))$. 

Powyższy układ równań możemy rozwiązać funkcją `rootSolve::multiroot`.

### Estymator IPW

Po uzyskaniu oszacowań (\hat{\mathbf{\theta})}) wyznaczamy ocenę estymatora *inverse probability weighted regression* (IPW) dany wzorem

$$
\hat{\mu}_{IPW1} = \frac{1}{N} \sum_{i \in S_A}\frac{y_i}{\hat{\pi}_i^A},
$$

oraz

$$
\hat{\mu}_{IPW2} = \frac{1}{\hat{N}} \sum_{i \in S_A}\frac{y_i}{\hat{\pi}_i^A},
$$

gdzie $\hat{N} = \sum_{i \in S_A} (\hat{\pi}_i^A)^{-1}$.


### Estymacja wariancji IPW2

Chen, Li i Wu (2020) wykazali, że wariancja estymatora IPW2 dany jest wzorem

\begin{equation}
V_{\mathrm{IPW} 2}=\frac{1}{N^{2}} \sum_{i=1}^{N}\left(1-\pi_{i}^{\mathrm{A}}\right) \pi_{i}^{\mathrm{A}}\left(\frac{y_{i}-\mu_{y}}{\pi_{i}^{\mathrm{A}}}-\mathbf{b}_{2}^{\top} \boldsymbol{x}_{i}\right)^{2}+\mathbf{b}_{2}^{\top} \mathbf{D} \mathbf{b}_{2},
\end{equation}

która w skrócie składa się z wariancji wynikajacej z próby nielosowej (przed symbolem +) oraz próby losowej (po symbolu +). Natomiast, w dalszej części wykazali, ze estymator ten można obliczyć następująco.


1. Cześć dotycząca wariancji wynikajacej z próby nielosowej.

\begin{equation}
\frac{1}{N^{2}} \sum_{i \in \mathcal{S}_{\mathrm{A}}}\left(1-\hat{\pi}_{i}^{\mathrm{A}}\right)\left(\frac{y_{i}-\hat{\mu}_{\mathrm{IPW} 2}}{\hat{\pi}_{i}^{\mathrm{A}}}-\hat{\mathbf{b}}_{2}^{\top} \boldsymbol{x}_{i}\right)^{2},
\end{equation}

gdzie 

\begin{equation}
\begin{aligned} 
\hat{\mathbf{b}}_{2}^{\top}=&\left\{\sum_{i \in \mathcal{S}_{\mathrm{A}}}\left(\frac{1}{\hat{\pi}_{i}^{\mathrm{A}}}-1\right)\left(y_{i}-\hat{\mu}_{\mathrm{IPW} 2}\right) \boldsymbol{x}_{i}^{\top}\right\} \times\left\{\sum_{i \in \mathcal{S}_{\mathrm{B}}} d_{i}^{\mathrm{B}} \hat{\pi}_{i}^{\mathrm{A}}\left(1-\hat{\pi}_{i}^{\mathrm{A}}\right) \boldsymbol{x}_{i} \boldsymbol{x}_{i}^{\top}\right\}^{-1} 
\end{aligned}
\end{equation}

2. Cześć wariancji wynikajaca z próby losowej (to wariancja wynikająca ze schematu losowania).

\begin{equation}
\hat{\mathbf{D}}=\frac{1}{N^{2}} \sum_{i \in \mathcal{S}_{\mathrm{B}} j \in \mathcal{S}_{\mathrm{B}}} \frac{\pi_{i j}^{\mathrm{B}}-\pi_{i}^{\mathrm{B}} \pi_{j}^{\mathrm{B}}}{\pi_{i j}^{\mathrm{B}}} \frac{\hat{\pi}_{i}^{\mathrm{A}}}{\pi_{i}^{\mathrm{B}}} \frac{\hat{\pi}_{j}^{\mathrm{A}}}{\pi_{j}^{\mathrm{B}}} \boldsymbol{x}_{i} \boldsymbol{x}_{j}^{\top}
\end{equation}



## Implementacja

Musimy rozwiązać następujacy układ równań:

$$
U(\mathbf{\theta}) = \frac{\partial \log L^*(\mathbf{\theta})}{\partial \mathbf{\theta}} = \sum_{i \in S_A} \mathbf{x}_i - \sum_{i \in S_B} d_i^B \pi(\mathbf{x}_i, \mathbf{\theta})\mathbf{x}_i.
$$

Oznacza to, że:

1. musimy określić zmienne $x$,
2. napisać funkcję, która przyjmie następujące parametry: $\theta$, $\mathbf{x}$ oraz $d$,
3. musimy również określić punkty startowe tj. $\theta_0$. W tym celu skorzystamy z funkcji `glm`,
4. do rozwiązania tego układu potrzebujemy funkcji `multiroot` z pakietu `rootSolve`.

Propozycja funkcji

```
propensity_score_Chen_Li_Wu_grad <- function(theta, x_a, x_b, d_b) {

  rho <- exp(x_b %*% theta) / (1 + exp(x_b %*% theta))
  
  ll_grad <- colSums(x_a) - colSums(d*rho*x_b)

  return(ll_grad)
}
```

Uwaga: macierze $x_a$ i $x_b$ będą macierzami rzadkimi (dużo zer, mało jedynek) dlatego warto rozważyć użycie wbudowanego pakietu `Matrix`.


Określamy punkty startowe wykorzystując funkcję `glm`. Ta funkcja posłuży również do określenia argumentów naszej funkcji tj. `x_a` i `x_b`.

In [168]:
m1 <- glm(proba ~ woj + wielk + sek + pkd, dane_estymacja, family = binomial, weights = waga)
summary(m1)


Call:
glm(formula = proba ~ woj + wielk + sek + pkd, family = binomial, 
    data = dane_estymacja, weights = waga)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-15.1136   -0.7098    1.5266    1.9084    2.7246  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -1.13143    0.07012 -16.135  < 2e-16 ***
woj4         0.59290    0.06493   9.131  < 2e-16 ***
woj6         0.22225    0.06806   3.265 0.001093 ** 
woj8         0.08578    0.08501   1.009 0.312927    
woj10       -0.10920    0.06162  -1.772 0.076382 .  
woj12       -0.44348    0.05809  -7.634 2.27e-14 ***
woj14       -0.66321    0.05202 -12.750  < 2e-16 ***
woj16        0.54342    0.07945   6.840 7.93e-12 ***
woj18        0.77734    0.06490  11.978  < 2e-16 ***
woj20        0.17500    0.08781   1.993 0.046274 *  
woj22       -0.01320    0.05762  -0.229 0.818800    
woj24       -0.29648    0.05255  -5.641 1.69e-08 ***
woj26        0.16736    0.08088   2.069 0.038522 *  
wo

Skorzystamy z obiektu `m1` i wywołamy w poniższym kodzie dwie funkcje `model.matrix` (zwróci macierz $\mathbf{X}$ dla obydwu zbiorów) oraz `coef`, która zwróci wektor parametrów z powyższego rozwiązania.

In [169]:
X <- model.matrix(m1) ## wyciągamy macierz X z obiektu m1
X_a <- X[dane_estymacja$proba == 1, ]
X_b <- X[dane_estymacja$proba == 0, ]
d_b <- dane_estymacja$waga[dane_estymacja$proba == 0] ## waga tylko dla tych rekordów z próby nielosowej
theta_0 <- coef(m1) ## parametry startowe
print(theta_0)

(Intercept)        woj4        woj6        woj8       woj10       woj12 
-1.13143144  0.59290290  0.22225067  0.08578418 -0.10919791 -0.44347549 
      woj14       woj16       woj18       woj20       woj22       woj24 
-0.66320954  0.54342174  0.77734499  0.17499654 -0.01320078 -0.29648025 
      woj26       woj28       woj30       woj32      wielkM      wielkS 
 0.16736498  0.41229717 -0.45433993  0.23647663 -0.74697007 -0.23289845 
       sek2      pkdD.E        pkdF        pkdG        pkdH        pkdI 
 0.05737559  0.50407033 -0.30484161 -0.26197458 -0.49829949  0.30314176 
       pkdJ      pkdK.L        pkdM        pkdN        pkdO        pkdP 
-1.20277711  0.06052943 -0.31623204  0.51655442 -0.47475660  0.76149170 
       pkdQ      pkdR.S 
 0.23515286  0.18382817 


In [170]:
propensity_score_Chen_Li_Wu_grad <- function(theta, x_a, x_b, d_b) {
 
  rho <- as.numeric(exp(x_b %*% theta) / (1 + exp(x_b %*% theta)))
 
  ll_grad <- colSums(x_a) - colSums(d_b*rho*x_b)
 
  return(ll_grad)
}


Sprawdzmy jak daleko od zera są punkty startowe z obiektu `theta_0`.

In [171]:
print(
  propensity_score_Chen_Li_Wu_grad(theta_0, X_a, X_b, d_b)
)

(Intercept)        woj4        woj6        woj8       woj10       woj12 
1875.512004  174.012954  104.016860   49.875420  100.292608   90.207448 
      woj14       woj16       woj18       woj20       woj22       woj24 
 134.169330   89.113284  191.146572   49.048079  151.190522  159.245438 
      woj26       woj28       woj30       woj32      wielkM      wielkS 
  56.020906  134.746546   97.247294  120.958287  504.538395  684.040271 
       sek2      pkdD.E        pkdF        pkdG        pkdH        pkdI 
1473.251074   67.737457  170.633843  289.727696   56.221557  132.361289 
       pkdJ      pkdK.L        pkdM        pkdN        pkdO        pkdP 
   3.717895   52.580060   41.109803  132.376037   44.435993  228.586429 
       pkdQ      pkdR.S 
 141.576530   66.314641 


Rozwiązujemy układ równań $U(\mathbf{\theta}) = \mathbf{0}$. Funkcja `multiroot` zwraca listę z następującymi argumentami 

+ `root` -- wektor parametrów $\hat{\mathbf{\theta}}$, który spełnia równianie $U(\hat{\mathbf{\theta}}) \approx \mathbf{0}$,
+ `f.root` -- jak blisko wartości zero jesteśmy,
+ `iter` -- po ilu iteracjach dochodzimy do celu

In [172]:
rsolve_res <- multiroot(f = propensity_score_Chen_Li_Wu_grad, 
                        start = theta_0, 
                        x_a = X_a, x_b = X_b, d_b = d_b)
print(rsolve_res)

$root
(Intercept)        woj4        woj6        woj8       woj10       woj12 
-0.65277121  0.83779550  0.19953188  0.10479761 -0.15756693 -0.60987005 
      woj14       woj16       woj18       woj20       woj22       woj24 
-0.84150300  0.76385615  1.17810496  0.22251032 -0.03753066 -0.40670702 
      woj26       woj28       woj30       woj32      wielkM      wielkS 
 0.20287478  0.57862243 -0.61021662  0.32742204 -1.02915884 -0.36412124 
       sek2      pkdD.E        pkdF        pkdG        pkdH        pkdI 
 0.05899377  0.77273741 -0.37783229 -0.33370352 -0.65174566  0.41179116 
       pkdJ      pkdK.L        pkdM        pkdN        pkdO        pkdP 
-1.42637011  0.06171389 -0.40677813  0.80034162 -0.69354695  1.25095429 
       pkdQ      pkdR.S 
 0.30286775  0.22227967 

$f.root
  (Intercept)          woj4          woj6          woj8         woj10 
-1.818989e-12 -2.273737e-13  5.684342e-14  0.000000e+00  1.136868e-13 
        woj12         woj14         woj16         woj18        

Wyznaczmy oszacowanie błędów standardowych dla wektora $\hat{\mathbf{\theta}}$ wykorzystując funkcję `rootSolve::gradient`.

In [173]:
solve_grad <- rootSolve::gradient(f = propensity_score_Chen_Li_Wu_grad, 
                                  x = rsolve_res$root, 
                                  x_a = X_a, x_b = X_b, d_b = d_b)

theta_se <- sqrt(abs(diag(solve(solve_grad))))

data.frame(theta = rsolve_res$root, theta_se = theta_se, t_value = rsolve_res$root/theta_se)

,theta,theta_se,t_value
,<dbl>,<dbl>,<dbl>
(Intercept),-0.65277121,0.07497595,-8.7064080
woj4,0.83779550,0.07121158,11.7648778
woj6,0.19953188,0.07245355,2.7539281
woj8,0.10479761,0.08910929,1.1760570
woj10,-0.15756693,0.06407774,-2.4589962
woj12,-0.60987005,0.06029341,-10.1150366
woj14,-0.84150300,0.05418685,-15.5296526
woj16,0.76385615,0.08659560,8.8209582
woj18,1.17810496,0.07142034,16.4953714


Dokonujemy estymacji wykorzystując wzór na IPW2.

In [175]:
nielosowa <- dane_estymacja %>% filter(proba == 1)

nielosowa <- nielosowa %>% 
    mutate(rho_Chen_Li_Wu = exp(as.numeric(X_a %*% rsolve_res$root))/(1 + exp(as.numeric(X_a %*% rsolve_res$root))),
           waga_Chen_Li_Wu = 1/rho_Chen_Li_Wu) 

ipw2_estym <- nielosowa %>%
  summarise(ipw2 = weighted.mean(odsetek, waga_Chen_Li_Wu))

ipw2_estym

ipw2
<dbl>
0.7215773


Interpretacja: wynik oznacza, że 72% wakatów dotyczy stanowisk na jedną zmianę.

### Estymacja wariancji

Wariancja estymatora IPW2 wynikająca z próby nielosowej dana jest

\begin{equation}
\frac{1}{N^{2}} \sum_{i \in \mathcal{S}_{\mathrm{A}}}\left(1-\hat{\pi}_{i}^{\mathrm{A}}\right)\left(\frac{y_{i}-\hat{\mu}_{\mathrm{IPW} 2}}{\hat{\pi}_{i}^{\mathrm{A}}}-\hat{\mathbf{b}}_{2}^{\top} \boldsymbol{x}_{i}\right)^{2},
\end{equation}

gdzie 

\begin{equation}
\begin{aligned} 
\hat{\mathbf{b}}_{2}^{\top}=&\left\{\sum_{i \in \mathcal{S}_{\mathrm{A}}}\left(\frac{1}{\hat{\pi}_{i}^{\mathrm{A}}}-1\right)\left(y_{i}-\hat{\mu}_{\mathrm{IPW} 2}\right) \boldsymbol{x}_{i}^{\top}\right\} \times\left\{\sum_{i \in \mathcal{S}_{\mathrm{B}}} d_{i}^{\mathrm{B}} \hat{\pi}_{i}^{\mathrm{A}}\left(1-\hat{\pi}_{i}^{\mathrm{A}}\right) \boldsymbol{x}_{i} \boldsymbol{x}_{i}^{\top}\right\}^{-1} 
\end{aligned}
\end{equation}



